In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import backend
# (3,100,100) -> channels first -> theano
# (100,100,3) -> channels last -> tensorflow 
from keras.layers.core import Dense, Dropout, Flatten, Activation 
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization 
from keras.models import Sequential


In [2]:
# Build a basic filtering block 
# Repeat filtering with larger/smaller filters 
#    filter window size-> always odd number
#     (1,1), (3,3), (5,5), -> image greater 128X128-> (3,3) or larger, use (1,1) on smaller
# you may not see larger than (7,7)
# keep compressing image/reducing dimensions 

In [3]:
# step to build CNN
# 1) build a basic filtering block-> outputs FeatureMaps (basic)
# 2) repeat advanced filtering blocks-> outputs more complex FeatureMaps 
# 3) Apply classification->   Flatten-> Dense(ReLu) -> Dense !!!


In [30]:
class randomCNN:
  @staticmethod
  def getModel(width, height, depth, classes, reg, init='he_normal'):
    model =Sequential()
    input_shape = (height, width, depth)
    chanDim = -1
    if backend.image_data_format() == "channels_first":
      input_shape = (depth, height, width)
      chanDim = 1
    
    #block
    convLayer1_1 = Conv2D(16, (7,7), strides=(2,2), padding="valid", kernel_initializer=init, kernel_regularizer=reg)
    convLayer2_1 = Conv2D(32, (3,3), padding="same", kernel_regularizer=reg, kernel_initializer=init )
    actLayer1_1 = Activation("relu")
    normLayer1_1 = BatchNormalization(axis=chanDim)
    convLayer3_1 = Conv2D(32, (3,3), strides=(2,2), padding="same", kernel_regularizer=reg, kernel_initializer=init )
    actLayer2_1 = Activation("relu")
    normLayer2_1 = BatchNormalization(axis=chanDim)
    regLayer1_1 = Dropout(0.25)
    block1 = [convLayer1_1, convLayer2_1, actLayer1_1,normLayer1_1, convLayer3_1, actLayer2_1, normLayer2_1, regLayer1_1]


    convLayer1_2 = Conv2D(64, (3,3), padding="same", kernel_regularizer=reg, kernel_initializer=init )
    actLayer1_2 = Activation("relu")
    normLayer1_2 = BatchNormalization(axis=chanDim)
    convLayer2_2 = Conv2D(64, (3,3), strides=(2,2), padding="same", kernel_regularizer=reg, kernel_initializer=init )
    actLayer2_2 = Activation("relu")
    normLayer2_2 = BatchNormalization(axis=chanDim)
    regLayer1_2 = Dropout(0.25)
    block2 = [convLayer1_2, actLayer1_2,normLayer1_2, convLayer2_2, actLayer2_2, normLayer2_2, regLayer1_2]
    

    convLayer1_3 = Conv2D(128, (3,3), padding="same", kernel_regularizer=reg, kernel_initializer=init )
    actLayer1_3 = Activation("relu")
    normLayer1_3 = BatchNormalization(axis=chanDim)
    convLayer2_3 = Conv2D(128, (3,3), strides=(2,2), padding="same", kernel_regularizer=reg, kernel_initializer=init )
    actLayer2_3 = Activation("relu")
    normLayer2_3 = BatchNormalization(axis=chanDim)
    regLayer1_3 = Dropout(0.25)
    block3 = [convLayer1_3, actLayer1_3,normLayer1_3, convLayer2_3, actLayer2_3, normLayer2_3, regLayer1_3]

    flattenLayer = Flatten()
    fullyConnectedLayer1 = Dense(512, kernel_initializer=init)
    actLayer1 = Activation("relu")
    normLayer1 = BatchNormalization()
    regLayer1 = Dropout(0.5)
    fullyConnectedLayer2 = Dense(classes)
    block_classification = [flattenLayer, fullyConnectedLayer1, actLayer1, normLayer1, regLayer1, fullyConnectedLayer2]
    allLayers = []
    allLayers.extend(block1)
    allLayers.extend(block2)
    allLayers.extend(block3)
    allLayers.extend(block_classification)
    #print(len(allLayers))
    model = Sequential(allLayers)
    
    return model



In [32]:
from tensorflow.keras import regularizers
model = randomCNN.getModel(96, 96, 3, 10, reg=regularizers.l2(0.0005))


In [36]:
# the final state of optimizer is -> loss = 0
# initial_learning rate when == 0, that implies we have arrived at most optimum point

# v = u + at 
# v = u - at 

# 0  =  1e-4 + decay_rate * epochs 
# -1e-4 = decay_rate * epochs
# -1e-4 / epochs = decay_rate
from tensorflow.keras.optimizers import Adam
init_lr = 1e-4
epochs = 50

mycustomOptimizer = Adam(lr= init_lr,decay=init_lr/epochs)


In [41]:
import matplotlib
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from imutils import paths

all_labels = set(["cat","dog"])


In [42]:
imagePaths = list(paths.list_images('data'))
imagePaths

['data/cat/cat1.jpg',
 'data/cat/cat2.jpg',
 'data/cat/3.png',
 'data/cat/cat3.jpg',
 'data/dog/d1.jpg',
 'data/dog/1.jpg',
 'data/dog/d2.jpg',
 'data/dog/d3.jpg']

In [43]:
data = []
labels= []

In [44]:
for impath in imagePaths:
  label = impath.split(os.path.sep)[-2]
  if label not in all_labels:
    continue
  image = cv2.imread(impath)
  image = cv2.resize(image, (96, 96))
  data.append(image)
  labels.append(label)

In [48]:
print(labels[0])


cat


In [49]:
#normalize images
data = np.array(data, dtype="float") / 255.0

lb = LabelBinarizer()
# outputs-> [0 1] -> [cat dog]-> key-value matrix
# 3 outs-> [0 1 0] -> [cat dog platypus]
# label encoding for 3 outputs-> 0 1 2 
# label encoding-> Animaltype = {cat, dog}-> cat=0, dog=1 -> value only

In [50]:
labels_transformed = lb.fit_transform(labels)
labels_transformed

array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1]])

In [51]:
xtrain, xtest, ytrain, ytest = train_test_split(data, labels_transformed, test_size=0.2, 
                                                stratify=labels_transformed, random_state=42)

In [52]:
# augmentation on data to train the model
# to generate sufficient training data

aug = ImageDataGenerator(rotation_range=40, shear_range=0.25, zoom_range=0.25, 
                         height_shift_range=0.2,
                         width_shift_range=0.2, fill_mode="nearest",
                         horizontal_flip=True, brightness_range=(0.5, 1.5))

In [59]:
batch_size = 1
epochs = 10
model.compile(loss='binary_crossentropy', optimizer=mycustomOptimizer, metrics=['accuracy'])
history = model.fit(x=aug.flow(xtrain, ytrain), 
                    validation_data=(xtest, ytest),
                    epochs= epochs)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 181ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 201ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 187ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 195ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 185ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 198ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 193ms/step -